## 1.提取DBC文件信息转为数据域分割的GroundTruth

In [1]:
import re
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
node = []
allDatas = []
siganlList = []
SignalsName = []
messageName = []

In [2]:
def readFile(file_name):
    global node,allDatas,siganlList,SignalsName,messageName
    ''' 得到dbc文件的绝对路径'''
    filePath = file_name
    if filePath:
        print(filePath)
        f = open(filePath, "r")  # 设置文件对象
    else:
        print("读取文件失败！")
        return 0
    """
    NodesPattern:节点
    MessagePattern：消息
    SignalPattern：信号
    """
    NodesPattern = re.compile(r"BU_: (.*)", re.S)
    MessagePattern = re.compile(r"BO_ (.*?) (.*?): (.*?) (.*)", re.S)
#     SignalPattern = re.compile('''SG_ (.*?) : (.*?)\|(.*?)@.*? \((.*?),(.*?)\) \[(.*?)\|(.*?)\] "(.*?)" (.*)''', re.S)
    SignalPattern = re.compile('''SG_ (.*?) : (.*?)\|(.*?)@([0-9])([+|-]) \((.*?),(.*?)\) \[(.*?)\|(.*?)\] "(.*?)" (.*)''', re.S)
    DefaultValue = '''BA_ "GenSig(.*?)" SG_ (\d+) signalname (\d+);'''

    line = f.readline()
    allDatas=[]
    while line:
        """ 匹配出节点 """
        NodesSearched = re.search(NodesPattern, line.strip())
        if NodesSearched:
            node = NodesSearched.group(1).split(" ")
            #print(node)
        """ 匹配出消息 """
        MessageSearched = re.search(MessagePattern, line.strip())
        if MessageSearched:
            siganlList.clear()
            """如果匹配到了message，则获取到message的相关参数 
             比如匹配到了NM_Message_ESC_409，则会解析出改message的一些参数构成list对象['1033', 'NM_Message_ESC_409', '8', 'ESC']
             这四个参数分别是 messgage ID ;message name ; messgae dataLenth ,message sender
             而且把这个list对象 加在了 siganlList 索引0的位置
            """
            Message = list(MessageSearched.groups())
            siganlList.append(Message)
            """ 只 要 message的名字 messageName 列表中"""
            messageName.append(Message[1])
            """读取下一行"""
            line = f.readline()
            """因为有些message并没有定义signal，所以 下一行还是message"""
            MessageSearched = re.search(MessagePattern, line.strip())
            SignalSearched = re.search(SignalPattern, line.strip())
            """下一行如果不是message的内容 就一定是signal的内容了"""
            if not MessageSearched:
                while SignalSearched:
                    """获取信号的参数追加到siganlList"""
                    signal = list(SignalSearched.groups())
                    siganlList.append(signal)
                    """只获取 signal name"""
                    SignalsName.append(signal[0])

                    # 再次解析信号，直到这个message下的信号全部解析完毕
                    line = f.readline()
                    SignalSearched = re.search(SignalPattern, line.strip())
           # print(siganlList)
            c = copy.deepcopy(siganlList)
            allDatas.append(c)
        else:
            line = f.readline()
            MessageSearched = re.search(MessagePattern, line.strip())
    f.close()  # 将文件关闭
    return 1

In [3]:
import os
file_names = []
path = os.path.abspath('../opendbc-master/opendbc-master-delFalseData')
# path = os.path.abspath('./opendbc-master/opendbc-master')
dirs = os.listdir(path)                    # 获取指定路径下的文件
for i in dirs:
    if os.path.splitext(i)[1] == ".dbc":
        file_names.append(os.path.join(path,i))
file_names

['D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master-delFalseData\\acura_ilx_2016_can_generated.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master-delFalseData\\acura_ilx_2016_nidec.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master-delFalseData\\acura_rdx_2018_can_generated.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master-delFalseData\\acura_rdx_2020_can_generated.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master-delFalseData\\bmw_e9x_e8x.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master-delFalseData\\cadillac_ct6_chassis.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\

## 2.处理原始bit数据和GroundTruth数据的函数

In [28]:
def get_id_labels(index):
    file_name = file_names[index]
    readFile(file_name)

    # 将DBC文件中的ID(10进制数)改为ID(16进制数)
    data_gts = {}
    for alldata in allDatas:
        data_gts[str(hex(int(alldata[0][0])))[2:]] = alldata

    # 得到DBC文件中各ID Message对应信号位置
    all_id_lsbs = {}
    for k in data_gts.keys():
        message = data_gts[k]
        lsb_list = []
        series = np.zeros(64)
        message_id = message[0][0]+"_"+message[0][1]
        del message[0]
        for signal in message:
            start = int(signal[1])
            length = int(signal[2])
            endianness = int(signal[3])
            start_row = int(start / 8)
            start_col = 8 - (start - start_row * 8) - 1
            new_start = start_row * 8 + start_col
            if endianness == 0:
                new_end = new_start + length
                cur_lsb = new_end - 1
            if endianness == 1:
                cur_lsb = new_start
            lsb_list.append(cur_lsb)
        all_id_lsbs[k] = lsb_list

    # 将各ID Message的信号位置改为 0/1 label
    all_id_labels = {}
    for k in all_id_lsbs:
        cur_lsb_list = all_id_lsbs[k]
        cur_label = np.zeros(64)
        for cur_lsb in cur_lsb_list:
            cur_label[cur_lsb] = 1
        all_id_labels[k] = cur_label
    return all_id_labels

In [29]:
def get_trace(data):
    group = data.groupby(by="ID")
    id_tracedict = {}
    for trace in list(group):
        id_tracedict[trace[0]] = trace[1]["bin"].apply(lambda x: list(x)).apply(lambda x: list(map(int, x))).tolist()
    return id_tracedict

In [30]:
def get_ids(id_tracedict_int, all_id_labels):
    ids = []
    for id in id_tracedict_int.keys():
        if(id_tracedict_int.__contains__(id) and all_id_labels.__contains__(id)):
            ids.append(id)
    return ids

In [17]:
import random
# 设置全局的随机种子
random.seed(100)
# 以id为单位shuffle
def get_train_test_ids(id_tracedict, all_id_labels):
    ids = get_ids(id_tracedict, all_id_labels)
    random.shuffle(ids)
    train_len = int(len(ids)*0.7)
    train_ids = ids[:train_len]
    test_ids = ids[train_len:]
    return train_ids, test_ids

In [18]:
def get_trace_data_dict(cur_ids, id_tracedict_int, all_id_labels, trace_data_dict, trace_label_dict): 
    for id in cur_ids:
        trace_data_dict[id] = id_tracedict_int[id]
        trace_label_dict[id] = all_id_labels[id]

## 3.综合Honda和Cadillac数据，得到训练集和测试集（包括0/1Label）

**在ID层面Shuffle**

In [27]:
honda_dbcs = []
cadillac_dbcs = []
tesla_dbcs = []
for i in range(len(file_names)):
    cur_name = file_names[i].split("\\")[-1].split("_")[0]
    if cur_name == "honda":
        honda_dbcs.append(i)
    if cur_name == "cadillac":
        cadillac_dbcs.append(i)
    if cur_name == "tesla":
        tesla_dbcs.append(i)

In [35]:
# d1 = {'name': 'revotu', 'age': 24}
# d2 = {'age': 25, 'sex': 'male'}
# d = {**d1, **d2}
cadillac_id_labels = {}
for cur_dbc_index in cadillac_dbcs:
    cadillac_id_labels.update(get_id_labels(cur_dbc_index)) 
print(len(cadillac_id_labels))  

D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\cadillac_ct6_chassis.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\cadillac_ct6_object.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\cadillac_ct6_powertrain.dbc
323


In [9]:
# 合并所有Cadillac以及Chevy车的Ground Truth的 0/1 label
dict0 = get_id_labels(0)
dict1 = get_id_labels(1)
dict2 = get_id_labels(2)
cadillac_id_labels = dict()
for key in dict0.keys() | dict1.keys() | dict2.keys():
    for d in (dict0, dict1, dict2):
        if d.__contains__(key):
            cadillac_id_labels[key] = d[key]
            break

D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\My_DBC\cadillac_ct6_object.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\My_DBC\cadillac_ct6_powertrain.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\My_DBC\gm_global_a_powertrain.dbc


In [10]:
# 合并所有Honda车的Ground Truth的 0/1 label
dict3 = get_id_labels(3)
dict4 = get_id_labels(4)
dict5 = get_id_labels(5)
honda_id_labels = dict()
for key in dict3.keys() | dict4.keys() | dict5.keys():
    for d in (dict3, dict4, dict5):
        if d.__contains__(key):
            honda_id_labels[key] = d[key]
            break

D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\My_DBC\honda_civic_hatchback_ex_2017_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\My_DBC\honda_civic_sedan_16_diesel_2019_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\My_DBC\honda_civic_touring_2016_can_generated.dbc


In [42]:
# 得到Honda车的子数据集数据
data_hd_1 = pd.read_csv('../Data/Honda/honda001.csv')
data_hd_2 = pd.read_csv('../Data/Honda/honda002.csv')
data_hd = pd.concat([data_hd_1,data_hd_2])
data_hd["bin"] = data_hd["Data"].apply(int, base=16).apply(bin).str[2:].apply(lambda x: x.zfill(64))

# 得到Honda车子的数据集的大Trace dict
id_tracedict_hd = get_trace(data_hd)

# 得到用于训练和测试的id
train_ids_hd, test_ids_hd = get_train_test_ids(id_tracedict_hd, honda_id_labels)
   
# shuffle后得到训练集和测试集，其中训练集的id占70%，测试集的id占30%

In [43]:
# 得到Cadillac车的子数据集数据
data_cd_1 = pd.read_csv('../Data/Cadillac/CadillacCSV/001.txt.csv')
data_cd_2 = pd.read_csv('../Data/Cadillac/CadillacCSV/002.txt.csv')
data_cd = pd.concat([data_cd_1,data_cd_2])
data_cd["bin"] = data_cd["Data"].apply(int, base=16).apply(bin).str[2:].apply(lambda x: x.zfill(64))

# 得到Honda车子的数据集的大Trace dict
id_tracedict_cd = get_trace(data_cd)

# 得到用于训练和测试的id
train_ids_cd, test_ids_cd = get_train_test_ids(id_tracedict_cd, cadillac_id_labels)
   
# shuffle后得到训练集和测试集，其中训练集的id占70%，测试集的id占30%

MemoryError: Unable to allocate 23.2 MiB for an array with shape (3042808,) and data type int64

In [ ]:
# 得到honda和caddilac车的trace dict及对应的label dict
train_data_dict = {}
train_label_dict = {}
get_trace_data_dict(train_ids_hd, id_tracedict_hd, honda_id_labels, train_data_dict, train_label_dict)
get_trace_data_dict(train_ids_cd, id_tracedict_cd, cadillac_id_labels, train_data_dict, train_label_dict)

test_data_dict = {}
test_label_dict = {}
get_trace_data_dict(test_ids_hd, id_tracedict_hd, honda_id_labels, test_data_dict, test_label_dict)
get_trace_data_dict(test_ids_cd, id_tracedict_cd, cadillac_id_labels, test_data_dict, test_label_dict)

## 4.CAN-D数据特征提取函数

In [17]:
def get_trace_data(id_tracedict_int, all_id_labels): 
    cur_trace_data = []
    cur_trace_label = []
    for id in id_tracedict_int.keys():
        cur_trace = id_tracedict_int[id]
        cur_label = all_id_labels[id]
        length = len(cur_trace)
        start = 4000
        end = start + 1000
        n = int((length-4000)/1000)
        for i in range(n):
            cur_trace_data.append(cur_trace[start: end])
            cur_trace_label.append(cur_label)
            start = end
            end = start + 1000
    return cur_trace_data, cur_trace_label

In [18]:
# 使用READ方法计算bit-flip
def get_bit_Flips(cur_id_traces):
    bit_Flips = []
    for i in range(len(cur_id_traces)):
        messagelist = cur_id_traces[i]
        trace_len = len(messagelist)
        bit_flip = np.zeros(64)
        previous = messagelist[0]
        for item in messagelist:
            for ix in range(64):
                if item[ix] != previous[ix]:
                    bit_flip[ix] = bit_flip[ix] + 1
            previous = item
        for ix in range(64):
            bit_flip[ix] = bit_flip[ix] / trace_len
        bit_flip = np.append(bit_flip, 0)
        bit_flip = np.append(bit_flip, 0)
        bit_Flips.append(bit_flip)
    return bit_Flips

In [27]:
# 使用READ方法计算bit-flip的联合概率
def get_bit_Flip_LHs(cur_id_traces):
    bit_Flip_LHs = []
    bit_Flip_LH_nos = []
    for i in range(len(cur_id_traces)):
        messagelist = cur_id_traces[i]
        trace_len = len(messagelist)
        bit_flip_LH = np.zeros(64)
        bit_flip_LH_no = np.zeros(64)
        previous_LH = messagelist[0]
        for item in messagelist:
            for ix in range(63):
                if item[ix] != previous_LH[ix] and item[ix+1] != previous_LH[ix+1]:
                    bit_flip_LH[ix] = bit_flip_LH[ix] + 1
                if item[ix] == previous_LH[ix] and item[ix+1] == previous_LH[ix+1]:
                    bit_flip_LH_no[ix] = bit_flip_LH_no[ix] + 1
            previous_LH = item
        for ix in range(63):
            bit_flip_LH[ix] = bit_flip_LH[ix] / trace_len
            bit_flip_LH_no[ix] = bit_flip_LH_no[ix] / trace_len

        bit_flip_LH = np.append(bit_flip_LH, 0)
        bit_flip_LH = np.append(bit_flip_LH, 0)
        bit_flip_LH_no = np.append(bit_flip_LH_no, 0)
        bit_flip_LH_no = np.append(bit_flip_LH_no, 0)

        bit_Flip_LHs.append(bit_flip_LH)
        bit_Flip_LH_nos.append(bit_flip_LH_no)
    return bit_Flip_LHs, bit_Flip_LH_nos

In [20]:
# reverse_id_tracelist 通过大端bit序列得到小端bit序列
def get_reverse_id_traces(cur_id_traces):
    reverse_id_traces = []
    for i in range(len(cur_id_traces)):
        id_tracelist = cur_id_traces[i]
        reverse_id_tracelist = []
        for item in id_tracelist:
            cur_tracelist = [0,1,2,3,4,5,6,7]
            cur_temp = []
            count = 7
            for i in range(64):
                cur_temp.append(item[i])
                if (i+1)%8 == 0:
                    cur_tracelist[count] = cur_temp
                    count = count - 1
                    cur_temp = []
            reverse_id_tracelist.append(np.concatenate(cur_tracelist).tolist())
        reverse_id_traces[i] = reverse_id_tracelist
    return reverse_id_traces

In [34]:
# 得到每个bit的15个特征
def get_sub_feature(bit_Flips, bit_Flips_LH, bit_Flip_LH_nos, ix, i):
    f1 = bit_Flips[i][ix]
    f2 = bit_Flips_LH[i][ix] / bit_Flips[i][ix+1]
    f3 = bit_Flips_LH[i][ix] / bit_Flips[i][ix]
    f4 = bit_Flip_LH_nos[i][ix] / (1 - bit_Flips[i][ix+1])
    f5 = bit_Flip_LH_nos[i][ix] / (1 - bit_Flips[i][ix])
    return np.array([f1,f2,f3,f4,f5])

def get_feature(bit_Flips, bit_Flips_LH, bit_Flip_LH_nos, ix, i):
    cur_bit_feature = []
    feature_sub1 = get_sub_feature(bit_Flips, bit_Flips_LH, bit_Flip_LH_nos, ix, i)
    feature_sub2 = get_sub_feature(bit_Flips, bit_Flips_LH, bit_Flip_LH_nos, ix+1, i)
    feature_sub3 = feature_sub2 - feature_sub1
    return np.concatenate((feature_sub1,feature_sub2,feature_sub3))

In [22]:
# 为每个bit赋予15个特征
def get_bit_features_list(bit_Flips, bit_Flips_LH, bit_Flip_LH_nos):
    bit_features_list = []
    for i in range(len(bit_Flips)):
        bit_features = []
        for ix in range(64):
            bit_features.append(get_feature(bit_Flips, bit_Flips_LH, bit_Flip_LH_nos, ix, i))
        bit_features_list.append(bit_features)
    return bit_features_list

In [23]:
# 小端转成大端的正常格式，方便与上边生成的ground truth label匹配
def LE_to_BE(reverse_bit_features_list):
    bit_features_list = []
    for i in range(len(reverse_bit_features_list)):
        item = reverse_bit_features_dict[i]
        cur_bit_features = [0,1,2,3,4,5,6,7]
        cur_temp = []
        count = 7
        for i in range(64):
            cur_temp.append(item[i])
            if (i+1)%8 == 0:
                cur_bit_features[count] = cur_temp
                count = count - 1
                cur_temp = []
        bit_features_list.append(np.concatenate(cur_bit_features).tolist())
    return bit_features_list

In [24]:
# 得到Trace与Ground Truth都存在的id
def get_all_datas_labels(bit_features_list, cur_id_labels):
    alldatas = []
    alllabels = []
    for i in range(len(bit_features_list)):
        cur_features = bit_features_list[i]
        cur_labels = cur_id_labels[i]
        for i in range(64):
            alldatas.append(cur_features[i])
            alllabels.append(int(cur_labels[i]))
    return alldatas, alllabels

In [25]:
def get_shuffle_data_label(cur_trace_data, cur_trace_label):
    trace_indexs = pd.DataFrame(cur_trace_label).index.tolist()
    random.shuffle(trace_indexs)
    trace_data, trace_labels = np.array(cur_trace_data)[trace_indexs], np.array(cur_trace_label)[trace_indexs]
    return trace_data, trace_labels

## 5.训练大端的RF模型

In [ ]:
train_id_traces, train_id_labels = get_trace_data(train_data_dict, train_label_dict)
bit_Flips_train = get_bit_Flips(train_id_traces)
bit_Flip_LHs_train, bit_Flip_LH_nos_train = get_bit_Flip_LHs(train_id_traces)
bit_features_list_train = get_bit_features_list(bit_Flips_train, bit_Flip_LHs_train, bit_Flip_LH_nos_train)
traindatas_msb, trainlabels_msb = get_all_datas_labels(bit_features_list_train, train_id_labels)

In [ ]:
test_id_traces, test_id_labels = get_trace_data(test_data_dict, test_label_dict)
bit_Flips_test = get_bit_Flips(test_id_traces)
bit_Flip_LHs_test, bit_Flip_LH_nos_test = get_bit_Flip_LHs(test_id_traces)
bit_features_list_test = get_bit_features_list(bit_Flips_test, bit_Flip_LHs_test, bit_Flip_LH_nos_test)
testdatas_msb, testlabels_msb = get_all_datas_labels(bit_features_list_test, test_id_labels)

In [ ]:
traindatas_msb, trainlabels_msb = get_shuffle_data_label(traindatas_msb, trainlabels_msb)
testdatas_msb, testlabels_msb = get_shuffle_data_label(testdatas_msb, testlabels_msb)

In [ ]:
Traindata_msb = pd.DataFrame(traindatas_msb)
Traindata_msb['label'] = np.array(trainlabels_msb)
Traindata_msb = Traindata_msb.fillna(0)
traindatas_msb = np.array(Traindata_msb.iloc[:,:15])
trainlabels_msb = np.array(Traindata_msb.iloc[:,-1])

In [ ]:
Testdata_msb = pd.DataFrame(testdatas_msb)
Testdata_msb['label'] = np.array(testlabels_msb)
Testdata_msb = Testdata_msb.fillna(0)
testdatas_msb = np.array(Testdata_msb.iloc[:,:15])
testlabels_msb = np.array(Testdata_msb.iloc[:,-1])

In [ ]:
#随机森林方法
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
model_msb = RandomForestClassifier(n_estimators= 30)
model_msb.fit(traindatas_msb, trainlabels_msb)

print("训练集:")
predict_target_msb = model_msb.predict(traindatas_msb)
print("预测正确数量,训练集样本量:")
print(sum(predict_target_msb == trainlabels_msb),len(trainlabels_msb))
print("训练集精确度等指标：")
print(metrics.classification_report(trainlabels_msb,predict_target_msb))
print("混淆矩阵：")
print(metrics.confusion_matrix(trainlabels_msb,predict_target_msb))

print("测试集:")
predict_target_test_msb =model_msb.predict(testdatas_msb)
print("预测正确数量,训练集样本量:")
print(sum(predict_target_test_msb == testlabels_msb),len(testlabels_msb))
print("测试集精确度等指标：")
print(metrics.classification_report(testlabels_msb,predict_target_test_msb))
print("混淆矩阵：")
print(metrics.confusion_matrix(testlabels_msb,predict_target_test_msb))

In [76]:
import joblib
# RF大端模型保存
joblib.dump(filename='./Model/CAN-D_FR_MSB.model', value=model_msb)

['./Model/CAN-D_FR_MSB.model']

## 训练小端的RF模型

In [68]:
reverse_train_data_dict = get_reverse_id_tracedict(train_data_dict)
reverse_bit_flip_dict_train = get_bit_flip_dict(reverse_train_data_dict)
reverse_bit_flip_LH_dict_train, reverse_bit_flip_LH_no_dict_train = get_bit_flip_LH_dict(reverse_train_data_dict)
reverse_bit_features_dict_train = get_bit_features_dict(reverse_bit_flip_dict_train, reverse_bit_flip_LH_dict_train, reverse_bit_flip_LH_no_dict_train)
bit_features_dict_train_lsb = LE_to_BE(reverse_bit_features_dict_train)
traindatas_lsb, trainlabels_lsb = get_all_datas_labels(bit_features_dict_train_lsb, train_label_dict)

C:\Users\Linever\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.
C:\Users\Linever\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [69]:
reverse_test_data_dict = get_reverse_id_tracedict(test_data_dict)
reverse_bit_flip_dict_test = get_bit_flip_dict(reverse_test_data_dict)
reverse_bit_flip_LH_dict_test, reverse_bit_flip_LH_no_dict_test = get_bit_flip_LH_dict(reverse_test_data_dict)
reverse_bit_features_dict_test = get_bit_features_dict(reverse_bit_flip_dict_test, reverse_bit_flip_LH_dict_test, reverse_bit_flip_LH_no_dict_test)
bit_features_dict_test_lsb = LE_to_BE(reverse_bit_features_dict_test)
testdatas_lsb, testlabels_lsb = get_all_datas_labels(bit_features_dict_test_lsb, test_label_dict)

C:\Users\Linever\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.
C:\Users\Linever\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [70]:
traindatas_lsb, trainlabels_lsb = get_shuffle_data_label(traindatas_lsb, trainlabels_lsb)
testdatas_lsb, testlabels_lsb = get_shuffle_data_label(testdatas_lsb, testlabels_lsb)

In [71]:
Traindata_lsb = pd.DataFrame(traindatas_lsb)
Traindata_lsb['label'] = np.array(trainlabels_lsb)
Traindata_lsb = Traindata_lsb.fillna(0)
traindatas_lsb = np.array(Traindata_lsb.iloc[:,:15])
trainlabels_lsb = np.array(Traindata_lsb.iloc[:,-1])

Testdata_lsb = pd.DataFrame(testdatas_lsb)
Testdata_lsb['label'] = np.array(testlabels_lsb)
Testdata_lsb = Testdata_lsb.fillna(0)
testdatas_lsb = np.array(Testdata_lsb.iloc[:,:15])
testlabels_lsb = np.array(Testdata_lsb.iloc[:,-1])

In [73]:
#随机森林方法
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
model_lsb = RandomForestClassifier(n_estimators= 30)
model_lsb.fit(traindatas_lsb, trainlabels_lsb)

print("训练集:")
predict_target_lsb = model_lsb.predict(traindatas_lsb)
print("预测正确数量,训练集样本量:")
print(sum(predict_target_lsb == trainlabels_lsb), len(trainlabels_lsb))
print("训练集精确度等指标：")
print(metrics.classification_report(trainlabels_lsb, predict_target_lsb))
print("混淆矩阵：")
print(metrics.confusion_matrix(trainlabels_lsb, predict_target_lsb))

print("测试集:")
predict_target_test_lsb = model_lsb.predict(testdatas_lsb)
print("预测正确数量,训练集样本量:")
print(sum(predict_target_test_lsb == testlabels_lsb),len(testlabels_lsb))
print("测试集精确度等指标：")
print(metrics.classification_report(testlabels_lsb, predict_target_test_lsb))
print("混淆矩阵：")
print(metrics.confusion_matrix(testlabels_lsb, predict_target_test_lsb))

训练集:
预测正确数量,训练集样本量:
2717 2944
训练集精确度等指标：
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      2644
           1       0.95      0.26      0.40       300

    accuracy                           0.92      2944
   macro avg       0.94      0.63      0.68      2944
weighted avg       0.93      0.92      0.90      2944

混淆矩阵：
[[2640    4]
 [ 223   77]]
测试集:
预测正确数量,训练集样本量:
1138 1280
测试集精确度等指标：
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      1142
           1       0.43      0.09      0.14       138

    accuracy                           0.89      1280
   macro avg       0.66      0.54      0.54      1280
weighted avg       0.85      0.89      0.85      1280

混淆矩阵：
[[1126   16]
 [ 126   12]]


In [77]:
# RF大端模型保存
joblib.dump(filename='./Model/CAN-D_FR_LSB.model', value=model_lsb)

['./Model/CAN-D_FR_LSB.model']

In [85]:
V = []
setlist1 = ['JB', 'C']
setlist2 = ['JB', 'JL', 'C']
setlist3 = ['JL', 'C']
flag = True
count = 0
for a in setlist1:
    for b in setlist2:
        for c in setlist2:
            for d in setlist2:
                for e in setlist2:
                    for f in setlist2:
                        for g in setlist2:
                            for h in setlist3:
                                v = [a,b,c,d,e,f,g,h]
                                for i in range(8):
                                    if(v[i] == 'JB' and v[i+1] == 'JL'):
                                        flag = False
                                        break
                                    if(v[i] == 'JB' and i<6 and v[i+2] == 'JL'):
                                        flag = False
                                        break
                                if flag == True:
                                    V.append(v)
                                    count = count + 1
                                flag = True

In [86]:
E = []
for v in V:
    e = ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']
    for i in range(8):
        if v[i] == 'JB':
            e[i] = 'B'
            e[i+1] = 'B'
        if v[i] == 'JL':
            e[i] = 'L'
            e[i-1] = 'L'
    E.append(e)

In [87]:
EB = pd.DataFrame(E).replace('C', 'B')
EB = np.array(EB).tolist()

EL = pd.DataFrame(E).replace('C', 'L')
EL = np.array(EL).tolist()

In [88]:
# 得到所有可能的信号标记集合T
def get_T(proba_msb, proba_lsb):
    T = []
    for e in EB:
        t = []
        for i in range(8):
            if e[i] == 'B':
                start = i * 8
                end = (i+1) * 8
                for j in range(start, end):
                    t.append(proba_msb[j])
            if e[i] == 'L':
                start = i * 8
                end = (i+1) * 8
                for j in range(start, end):
                    t.append(proba_lsb[j])
        T.append(t)
    for e in EL:
        t = []
        for i in range(8):
            if e[i] == 'B':
                start = i * 8
                end = (i+1) * 8
                for j in range(start, end):
                    t.append(proba_msb[j])
            if e[i] == 'L':
                start = i * 8
                end = (i+1) * 8
                for j in range(start, end):
                    t.append(proba_lsb[j])
        T.append(t)
    return T

In [89]:
predict_proba_msb_dict = {}
for id in bit_features_dict_test.keys():
    cur_predictproba_msb = []
    temp = pd.DataFrame(bit_features_dict_test[id]).fillna(0)
    cur_predict_proba_msb = model_msb.predict_proba(temp)
    for prediction in cur_predict_proba_msb:
        cur_predictproba_msb.append(prediction[1])
    predict_proba_msb_dict[id] = cur_predictproba_msb
    
predict_proba_lsb_dict = {}
for id in bit_features_dict_test_lsb.keys():
    cur_predictproba_lsb = []
    temp = pd.DataFrame(bit_features_dict_test_lsb[id]).fillna(0)
    cur_predict_proba_lsb = model_lsb.predict_proba(temp)
    for prediction in cur_predict_proba_lsb:
        cur_predictproba_lsb.append(prediction[1])
    predict_proba_lsb_dict[id] = cur_predictproba_lsb

In [90]:
B = 0.6
final_label_dict = {}
for id in predict_proba_msb_dict.keys():
    curT = get_T(predict_proba_msb_dict[id], predict_proba_lsb_dict[id])
    mint0 = []
    for t in curT:
        t0 = 0
        for f in t:
            t0 = t0 + min(f, B)
        mint0.append(t0)
    tf = pd.DataFrame(curT)
    tf['t0'] = mint0
    res = tf[tf['t0'] == min(mint0)]
    final_res = res.drop_duplicates()
    final_t = np.array(final_res)[0][:-1]
    final_label = []
    for i in range(64):
        if final_t[i] >= B:
            final_label.append(i)
    final_label_dict[id] = final_label

In [111]:
test_label_locate_dict = {}
for id in test_label_dict.keys():
    temp_list = test_label_dict[id]
    cur_label = []
    for i in range(64):
        if(temp_list[i] == 1):
            cur_label.append(i)
    test_label_locate_dict[id] = cur_label

In [113]:
for id in final_label_dict.keys():
    print(final_label_dict[id])
    print(test_label_locate_dict[id])
    print("------------------------------------------")

[59, 63]
[7, 31, 32, 33, 38, 39, 49, 50, 55, 59, 63]
------------------------------------------
[59]
[15, 23, 31, 47, 49, 59, 63]
------------------------------------------
[]
[40, 41, 42, 49, 51, 59, 63]
------------------------------------------
[31, 59]
[15, 31, 59, 63]
------------------------------------------
[63]
[15, 27, 41, 42, 59, 63]
------------------------------------------
[]
[]
------------------------------------------
[]
[63]
------------------------------------------
[39]
[15, 31]
------------------------------------------
[]
[5, 6, 7, 14, 23, 50, 52, 63]
------------------------------------------
[48]
[31, 32, 34, 47]
------------------------------------------
[15]
[55]
------------------------------------------
[]
[3, 4, 15, 31]
------------------------------------------
[]
[55]
------------------------------------------
[]
[23, 39]
------------------------------------------
[23, 52]
[0, 1, 3, 4, 23, 31, 52, 63]
------------------------------------------
[59]
[16, 1

In [93]:
res_test_label_dict = {}
for id in final_label_dict.keys():
    cur_label = np.zeros(64)
    for location in final_label_dict[id]:
        cur_label[location] = 1
    res_test_label_dict[id] = cur_label

In [101]:
def get_flatten_array(label_dict):
    flatten_list = []
    for id in label_dict.keys():
        bitlist = label_dict[id]
        for i in range(64):
            flatten_list.append(bitlist[i])
    return np.array(flatten_list)

In [102]:
res = get_flatten_array(res_test_label_dict)
labels = get_flatten_array(test_label_dict)

In [103]:
# 每个bit是否识别正确的准确度统计

print("预测正确数量,训练集样本量:")
print(sum(res == labels),len(labels))
print("训练集精确度等指标：")
print(metrics.classification_report(labels,res))
print("混淆矩阵：")
print(metrics.confusion_matrix(labels,res))

预测正确数量,训练集样本量:
1146 1280
训练集精确度等指标：
              precision    recall  f1-score   support

         0.0       0.90      1.00      0.94      1142
         1.0       0.67      0.06      0.11       138

    accuracy                           0.90      1280
   macro avg       0.78      0.53      0.53      1280
weighted avg       0.87      0.90      0.85      1280

混淆矩阵：
[[1138    4]
 [ 130    8]]


In [109]:
# lsb边界是否识别正确的准确度统计

right_count = 0
total_count = 0
for i in range(len(labels)):
    if(labels[i] == 1):
        total_count = total_count + 1
        if(res[i] == 1):
            right_count = right_count + 1
acc = right_count / total_count

In [110]:
acc

0.057971014492753624